In [4]:
import json


count = {}
count_length = {}
sign_groups = {}


def create_index(
    index_list_to_use,
    hand,
    shape,
    hand_orientation,
    palm_orientation,
    fingers_orientation,
    location,
    movement,
):
    index = hand + "("

    if "shape" in index_list_to_use:
        index += f"{shape},"

    if "hand_orientation" in index_list_to_use:
        index += f"{hand_orientation},"

    if "palm_orientation" in index_list_to_use:
        index += f"{palm_orientation},"

    if "fingers_orientation" in index_list_to_use:
        index += f"{fingers_orientation},"

    if "location" in index_list_to_use:
        index += f"{location},"

    if "movement" in index_list_to_use:
        index += f"{movement},"

    return index + ")"


def load_json(filename, index_list_to_use, length):
    with open(filename) as f:
        data = json.load(f)

        for sign in data:
            try:
                content = sign["content"]
                indexes = ""
                count_length[len(content)] = count_length.get(len(content), 0) + 1
                for c in content[:length]:
                    shape = c.get("right", {}).get("shape", "default")
                    hand_orientation = c.get("right", {}).get("hand_orientation", "any")
                    palm_orientation = c.get("right", {}).get(
                        "palm_orientation", "default"
                    )
                    fingers_orientation = c.get("right", {}).get(
                        "fingers_orientation", "any"
                    )
                    location = c.get("right", {}).get("location", "chest")
                    right_movement = c.get("right", {}).get("movement", [])
                    movement = len(right_movement) > 0 and right_movement[0] or "nil"

                    shape_left = c.get("left", {}).get("shape", "default")
                    hand_orientation_left = c.get("left", {}).get(
                        "hand_orientation", "any"
                    )
                    palm_orientation_left = c.get("left", {}).get(
                        "palm_orientation", "default"
                    )
                    fingers_orientation_left = c.get("left", {}).get(
                        "fingers_orientation", "any"
                    )
                    location_left = c.get("left", {}).get("location", "default")
                    left_movement = c.get("left", {}).get("movement", [])
                    movement_left = len(left_movement) > 0 and left_movement[0] or "nil"

                    index_right = create_index(
                        index_list_to_use,
                        "r",
                        shape,
                        hand_orientation,
                        palm_orientation,
                        fingers_orientation,
                        location,
                        movement,
                    )
                    index_left = create_index(
                        index_list_to_use,
                        "l",
                        shape_left,
                        hand_orientation_left,
                        palm_orientation_left,
                        fingers_orientation_left,
                        location_left,
                        movement_left,
                    )
                    indexes += index_right + index_left

                count[indexes] = count.get(indexes, 0) + 1

                if indexes not in sign_groups:
                    sign_groups[indexes] = [sign]
                else:
                    sign_groups[indexes].append(sign)
            except Exception as e:
                print(e)

In [9]:
group = (
    "r(open,horizontal,left,any,chest,right,)l(open,horizontal,right,any,default,left,)"
)

print(
    json.dumps(
        sign_groups[group],
        ensure_ascii=False,
        indent=4,
    )
)

[
    {
        "id": "AMPLIAR",
        "content": [
            {
                "right": {
                    "shape": "open",
                    "hand_orientation": "horizontal",
                    "palm_orientation": "left",
                    "movement": [
                        "right"
                    ]
                },
                "left": {
                    "shape": "open",
                    "hand_orientation": "horizontal",
                    "palm_orientation": "right",
                    "movement": [
                        "left"
                    ]
                }
            }
        ]
    },
    {
        "id": "AMPLO",
        "content": [
            {
                "right": {
                    "shape": "open",
                    "hand_orientation": "horizontal",
                    "palm_orientation": "left",
                    "movement": [
                        "right"
                    ]
                },
                "lef

In [3]:
from PIL import Image
import os


def find_text_image(text):
    for text_path in sorted(os.listdir("texts")):
        if text in open(f"texts/{text_path}", "r").read():
            return text_path


def open_image(image_path):
    try:
        img = Image.open(image_path)
        img.show()
    except IOError:
        print(
            "Error: Unable to open image. Make sure the path is correct and you have permission to access the file."
        )


founded_path = find_text_image("CABEÇA (1)")
image_path = f"images/{founded_path.replace('.txt', '')}.png"

open_image(image_path)